In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
cookies = {
    'PHPSESSID': 'k321v1aaa2580l8u73vmjnqjqn',
    '_gid': 'GA1.3.984971739.1670923045',
    '_ga': 'GA1.1.2064957005.1665472402',
    '_ga_TNJL30HVP1': 'GS1.1.1670929678.4.1.1670929739.0.0.0',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'PHPSESSID=k321v1aaa2580l8u73vmjnqjqn; _gid=GA1.3.984971739.1670923045; _ga=GA1.1.2064957005.1665472402; _ga_TNJL30HVP1=GS1.1.1670929678.4.1.1670929739.0.0.0',
    'Referer': 'https://verification.fda.gov.ph/medical_deviceslist.php',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Google Chrome";v="108"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

params = {
    'recperpage': 'ALL',
}

response = requests.get(
    'https://verification.fda.gov.ph/medical_deviceslist.php',
    params=params,
    cookies=cookies,
    headers=headers,
    verify=False
)

C:\Users\AbhishekSrivastava\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'verification.fda.gov.ph'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
soup = BeautifulSoup(response.text)

In [4]:
devices = soup.find('table', id="tbl_medical_deviceslist").find('tbody')

In [5]:
devices = devices.find_all('tr')

In [6]:
devices

[<tr data-rowindex="1" data-rowtype="1" id="r1_medical_devices">
 <td class="ew-list-option-body text-nowrap" data-name="view"><span class="medical_devices_view" id="el1_medical_devices_view"><a class="ew-row-link ew-view" data-caption="View" href="medical_devicesview.php?showdetail=&amp;registration_number=CDMR-2021-00179" title="View">View</a></span></td> <td data-name="registration_number">
 <span id="el1_medical_devices_registration_number">
 <span>CDMR-2021-00179</span>
 </span>
 </td>
 <td data-name="product_name">
 <span id="el1_medical_devices_product_name">
 <span>Tron FX II Thrombectomy Device</span>
 </span>
 </td>
 <td data-name="manufacturer">
 <span id="el1_medical_devices_manufacturer">
 <span>Biomedical Solutions, Inc. Tokyo, Japan Mfd. By: Yokowo Co. Ltd. Tomoika Plant, - Gunma, Japan For: Biomedical Solutions Inc. - Tokyo, Japan</span>
 </span>
 </td>
 <td data-name="country_of_origin">
 <span id="el1_medical_devices_country_of_origin">
 <span>Japan</span>
 </span>
 <

In [7]:
cake = pd.DataFrame()
for device in devices:
    datapoints = device.find_all('td')
    row = []
    for datapoint in datapoints:
        if datapoint.find('a', href=True):
            value = "https://verification.fda.gov.ph/"+datapoint.find('a', href=True)['href']
        else:
            value = datapoint.text.strip()
        row.append(value)
    row = pd.DataFrame(row)
    row = row.transpose()
    cake = pd.concat([cake,row])

In [8]:
cake.shape

(18862, 7)

In [9]:
cake.columns = ['URL','Registration ID','Product Name','Manufacturer','Country of Origin','Issuance Date', 'Expiry Date']
cake = cake[['Registration ID','Product Name','Manufacturer','Country of Origin','Issuance Date', 'Expiry Date', 'URL']]
cake.to_excel("FDA PH_110124.xlsx")
cake.to_csv("FDA PH_110124.csv")